### 1. Introduction

Start with a title and brief project overview.
Clearly state the problem or question you're addressing.
Define the objectives and scope of the analysis.

### 2. Problem/Data Description

Provide detailed information about the dataset.
Explain the data source, size, format, and any relevant background information.
Describe the variables and their meaning.
Define the target variable if it's a supervised learning task.

### 3. Exploratory Data Analysis (EDA)

Import necessary libraries and load the dataset.
Summarize the data with basic statistics (mean, median, standard deviation).
Visualize the data to gain insights:
Histograms, box plots, scatter plots, etc.
Explore data distributions, correlations, and potential outliers.
Handle missing data and perform data preprocessing:
Data cleaning, feature engineering, and encoding categorical variables.

### 4. Data Preprocessing

Describe in detail how you handled data preprocessing tasks.
Explain any transformations, feature scaling, or encoding techniques used.

### 5. Model Building and Training

Define the problem type (e.g., classification, regression) and evaluation metrics.
Split the dataset into training, validation, and test sets.
Choose appropriate machine learning algorithms or models.
Set up the models with hyperparameters and configurations.
Train the models on the training data.
Perform hyperparameter tuning, if applicable.
Evaluate model performance on the validation set using chosen metrics.
Discuss any regularization or optimization techniques applied.

### 6. Results

Present the results of your analysis and model training.
Display relevant metrics (e.g., accuracy, F1 score, RMSE) for each model.
Use visualizations (e.g., confusion matrices, ROC curves) to illustrate model performance.
Discuss the findings and insights gained from the analysis.

### 7. Discussion/Conclusion

Summarize the key findings and results.
Interpret the results in the context of the problem statement.
Address any limitations or challenges encountered during the analysis.
Offer suggestions for further research or improvements.
Conclude with a concise summary of the project's significance and outcomes

### Problem and Data Description

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import re
import os
import random
import cv2
import shutil
import PIL

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization


import matplotlib.pyplot as plt
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

AUTOTUNE = tf.data.experimental.AUTOTUNE
    
print(tf.__version__)
# define path to data
GCS_PATH = KaggleDatasets().get_gcs_path()
PATH = '../input/gan-getting-started/'
MONET_TFREC = tf.io.gfile.glob(str(GCS_PATH + '/monet_tfrec/*.tfrec'))
PHOTO_TFREC = tf.io.gfile.glob(str(GCS_PATH + '/photo_tfrec/*.tfrec'))
MONET_JPG_PATH = os.path.join(PATH, 'monet_jpg/')
PHOTO_JPG_PATH = os.path.join(PATH, 'photo_jpg/')

### EDA

In [ ]:
def count_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) 
         for filename in filenames]
    return np.sum(n)


print(f'Monet TFRecord Files: {len(MONET_TFREC)}')
print(f'Monet images: {count_items(MONET_TFREC)}')

print('Photo TFRecord Files:', len(PHOTO_TFREC))
print(f'Photo images: {count_items(PHOTO_TFREC)}')
# decode and scale images
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = (tf.cast(image, tf.float32) / 255.0) - 1
    image = tf.reshape(image, [256, 256, 3])
    return image

def read_tfrecord(example):
    tfrecord_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image
# extract the image from the files
def load_dataset(filenames, labeled=True, ordered=False):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
    return dataset
monet_ds = load_dataset(MONET_TFREC, labeled=True).batch(1)
photo_ds = load_dataset(PHOTO_TFREC, labeled=True).batch(1)
monet_imgs = os.listdir(MONET_JPG_PATH)
photo_imgs = os.listdir(PHOTO_JPG_PATH)
# visualizing samples from Monet and photo datasets
fig, ax = plt.subplots(2, 2, figsize=(8, 8))
fig.tight_layout(pad=1.5)
for i in range(2):
    image = cv2.imread(os.path.join(MONET_JPG_PATH, random.choice(monet_imgs)))
    image2 = cv2.imread(os.path.join(PHOTO_JPG_PATH, random.choice(photo_imgs)))
    ax[i,0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    ax[i,0].set_title('Monet')
    ax[i,1].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
    ax[i,1].set_title('Photo')

In [ ]:
def plot_histogram(image, name):
    plt.figure(figsize=(40,10))
    plt.subplot(1,3,1)

    plt.axis('off')
    plt.title(name, fontsize=30)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
 
    blue_histogram = cv2.calcHist([image], [0], None, [256], [0, 256])
    red_histogram = cv2.calcHist([image], [1], None, [256], [0, 256])
    green_histogram = cv2.calcHist([image], [2], None, [256], [0, 256]) 
 
    plt.subplot(1,3,2)
    plt.title('Histogram of All Colors', fontsize=30)
    plt.hist(blue_histogram, color='darkblue')
    plt.hist(green_histogram, color='green')
    plt.hist(red_histogram, color='red')
 
    plt.subplot(1,3,3)
    plt.title('Line Plots of All Colors', fontsize=30)
    plt.plot(blue_histogram, color='darkblue')
    plt.plot(green_histogram, color='green')
    plt.plot(red_histogram, color='red')
 
    plt.tight_layout()
    plt.show()
    
image = cv2.imread(os.path.join(MONET_JPG_PATH, random.choice(monet_imgs)))
plot_histogram(image, name="monet image")
image2 = cv2.imread(os.path.join(PHOTO_JPG_PATH, random.choice(photo_imgs)))
plot_histogram(image2, name="photo image")